## Codigo Prueba Pratica 2

In [4]:
from docplex.mp.model import Model
import docplex.mp.solution as Solucion
import csv

Método que limpia los datos

In [5]:
def clean(row):
    nrow = []
    for i in row:
        #nrow.append(i)
        i = i.split("km")[0]
        nrow.append(float(i)) # Se cambia el tipo de dato
    return nrow

Método que lee el archivo CSV

In [6]:
def read_file(src):
    rows = []
    with open(src, 'r') as file:
        reader = csv.reader(file, delimiter = ';')
        c = 1
        for row in reader:
            if c==1:
                row[0] = '0' #Reemplaza el primer valor por un 0
                nrow = clean(row) # se limpian los datos
                #print (nrow)
            c+=1
            nrow = clean(row)
            rows.append(nrow)
    return rows

Método que multiplica las distancias en km por el costo total de pelos de fibra que se instalaran

In [7]:
def total_cost(datos):
    data_cost =[]
    for row in range(len(datos)):
        r = []
        for col in range(len(datos)):
            cost = datos[row][col] *202.8125 # ditancia * costo de cada pelo fibra km*dolar/(gbps*km)  ===> valor gbps
            r.append(cost)
        data_cost.append(r)
    return data_cost

Método que resuelve el problema

In [8]:
def modelo(mdl, Costos, plantas, destinos, Arcos, Capacidad, Demanda):

    #Variable entera de cantidad a producir
    x=mdl.continuous_var_dict(Arcos,name='x')
    #Función objetivo
    mdl.minimize(mdl.sum(x[i, j]*Costos.get((i, j)) for i in plantas for j in destinos if j!=i))  # funcion obj

    # Primero restricción de capacidad
    for k in plantas:
        mdl.add_constraint(mdl.sum(x[(k,j)] for j in Demanda if j!=k)<=Capacidad[k])

    # Segundo restricción de demanda
    for k in Demanda:
        mdl.add_constraint(mdl.sum(x[(i,k)] for i in Capacidad if i!=k)>=Demanda[k])

    print(mdl.export_to_string())

    solucion = mdl.solve(log_output=True)
    mdl.get_solve_status()
    return solucion.display()


datos = read_file('./data/data.csv')
aCost = total_cost(datos)

plantas=[k for k in range(20)]
destinos=[k for k in range(20)]
Arcos =[(i,j) for i in plantas for j in destinos]

Creando los diccionarios a utilizar en el modelo

In [9]:
Capacidad = {0: 0.03 * 147577,  # 147577
              1: 0.03 * 112449,  # 112449,
              2: 0.03 * 71258,
              3: 0.03 * 4397,
              4: 0.03 * 25947,
              5: 0.03 * 112449,  # 112449
              6: 0.03 * 31650,
              7: 0.03 * 6570,
              8: 0.03 * 20524,
              9: 0.03 * 16244,
              10: 0.03 * 6132,
              11: 0.03 * 7713,
              12: 0.03 * 19865,
              13: 0.03 * 16742,
              14: 0.03 * 3727,
              15: 0.03 * 8626,
              16: 0.03 * 4510,
              17: 0.03 * 7896,
              18: 0.03 * 1973,
              19: 0.03 * 1489
              }
Demanda = {
    0: 0.01 * 147577,  # 147577
    1: 0.01 * 112449,  # 112449
    2: 0.01 * 71258,
    3: 0.01 * 4397,
    4: 0.01 * 25947,
    5: 0.01 * 112449,  # 112449
    6: 0.01 * 31650,
    7: 0.01 * 6570,
    8: 0.01 * 20524,
    9: 0.01 * 16244,
    10: 0.01 * 6132,
    11: 0.01 * 7713,
    12: 0.01 * 19865,
    13: 0.01 * 16742,
    14: 0.01 * 3727,
    15: 0.01 * 8626,
    16: 0.01 * 4510,
    17: 0.01 * 7896,
    18: 0.01 * 1973,
    19: 0.01 * 1489
}
Costos = {(i, j): aCost[i][j] for i in range(len(aCost)) for j in range(len(aCost)) if j!=i}
mdl=Model('Transp')
modelo(mdl,Costos, plantas, destinos, Arcos, Capacidad, Demanda)


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Transp

Minimize
 obj: 4705.250000000000 x_0_1 + 5739.593750000000 x_0_2
      + 4522.718750000000 x_0_3 + 6997.031250000000 x_0_4
      + 6753.656249999999 x_0_5 + 4705.250000000000 x_0_6
      + 5435.375000000000 x_0_7 + 5536.781250000000 x_0_8
      + 7118.718750000000 x_0_9 + 6997.031250000000 x_0_10
      + 9167.125000000000 x_0_11 + 10282.593750000000 x_0_12
      + 12939.437500000000 x_0_13 + 10384 x_0_14 + 7321.531250000000 x_0_15
      + 10404.281250000000 x_0_16 + 9613.312500000000 x_0_17
      + 12493.250000000000 x_0_18 + 10972.156250000000 x_0_19
      + 4705.250000000000 x_1_0 + 7463.499999999999 x_1_2
      + 4725.531250000000 x_1_3 + 3103.031250000000 x_1_4
      + 10850.468750000000 x_1_5 + 2352.625000000000 x_1_6
      + 3204.437500000000 x_1_7 + 6469.718750000000 x_1_8
      + 7889.406250000000 x_1_9 + 5962.687500000000 x_1_10
      + 11215.531250000000 x_1_11 + 12351.281250000000 x_1_12
 

Método que realiza el análisis de sensibilidad

In [10]:
n_restrict = mdl.number_of_constraints
restricts = [mdl.get_constraint_by_index(i) for i in range(n_restrict)]
print(restricts)
#Variables de holgura
holguras = mdl.slack_values(restricts)
print(holguras)

[docplex.mp.LinearConstraint[](x_0_1+x_0_2+x_0_3+x_0_4+x_0_5+x_0_6+x_0_7+x_0_8+x_0_9+x_0_10+x_0_11+x_0_12+x_0_13+x_0_14+x_0_15+x_0_16+x_0_17+x_0_18+x_0_19,LE,4427.3099999999995), docplex.mp.LinearConstraint[](x_1_0+x_1_2+x_1_3+x_1_4+x_1_5+x_1_6+x_1_7+x_1_8+x_1_9+x_1_10+x_1_11+x_1_12+x_1_13+x_1_14+x_1_15+x_1_16+x_1_17+x_1_18+x_1_19,LE,3373.47), docplex.mp.LinearConstraint[](x_2_0+x_2_1+x_2_3+x_2_4+x_2_5+x_2_6+x_2_7+x_2_8+x_2_9+x_2_10+x_2_11+x_2_12+x_2_13+x_2_14+x_2_15+x_2_16+x_2_17+x_2_18+x_2_19,LE,2137.74), docplex.mp.LinearConstraint[](x_3_0+x_3_1+x_3_2+x_3_4+x_3_5+x_3_6+x_3_7+x_3_8+x_3_9+x_3_10+x_3_11+x_3_12+x_3_13+x_3_14+x_3_15+x_3_16+x_3_17+x_3_18+x_3_19,LE,131.91), docplex.mp.LinearConstraint[](x_4_0+x_4_1+x_4_2+x_4_3+x_4_5+x_4_6+x_4_7+x_4_8+x_4_9+x_4_10+x_4_11+x_4_12+x_4_13+x_4_14+x_4_15+x_4_16+x_4_17+x_4_18+x_4_19,LE,778.41), docplex.mp.LinearConstraint[](x_5_0+x_5_1+x_5_2+x_5_3+x_5_4+x_5_6+x_5_7+x_5_8+x_5_9+x_5_10+x_5_11+x_5_12+x_5_13+x_5_14+x_5_15+x_5_16+x_5_17+x_5_18+x_5_19,L

Mostrando variables de Holgura por cada restricción

In [11]:
for index in range(n_restrict):
    print("La variable de holgura de la restricción " + str(restricts[index]) + " es " + str(holguras[index]))

#Precios duales
precio_dual = mdl.dual_values(restricts)

# Mostrando variables de Holgura por cada restricción
for index in range(n_restrict):
    print("El precio dual de la restricción " + str(restricts[index]) + " es " + str(precio_dual[index]))

La variable de holgura de la restricción x_0_1+x_0_2+x_0_3+x_0_4+x_0_5+x_0_6+x_0_7+x_0_8+x_0_9+x_0_10+x_0_11+x_0_12+x_0_13+x_0_14+x_0_15+x_0_16+x_0_17+x_0_18+x_0_19 <= 4427.3099999999995 es 4427.3099999999995
La variable de holgura de la restricción x_1_0+x_1_2+x_1_3+x_1_4+x_1_5+x_1_6+x_1_7+x_1_8+x_1_9+x_1_10+x_1_11+x_1_12+x_1_13+x_1_14+x_1_15+x_1_16+x_1_17+x_1_18+x_1_19 <= 3373.47 es 1968.2899999999997
La variable de holgura de la restricción x_2_0+x_2_1+x_2_3+x_2_4+x_2_5+x_2_6+x_2_7+x_2_8+x_2_9+x_2_10+x_2_11+x_2_12+x_2_13+x_2_14+x_2_15+x_2_16+x_2_17+x_2_18+x_2_19 <= 2137.74 es 1036.0299999999997
La variable de holgura de la restricción x_3_0+x_3_1+x_3_2+x_3_4+x_3_5+x_3_6+x_3_7+x_3_8+x_3_9+x_3_10+x_3_11+x_3_12+x_3_13+x_3_14+x_3_15+x_3_16+x_3_17+x_3_18+x_3_19 <= 131.91 es 0
La variable de holgura de la restricción x_4_0+x_4_1+x_4_2+x_4_3+x_4_5+x_4_6+x_4_7+x_4_8+x_4_9+x_4_10+x_4_11+x_4_12+x_4_13+x_4_14+x_4_15+x_4_16+x_4_17+x_4_18+x_4_19 <= 778.41 es 294.6999999999998
La variable de holg

Reporte de Sensibilidad, como se obtiene la sensibilidad del lado derecho o de la función objetivo en Cplex.

In [12]:
cpx = mdl.get_engine().get_cplex()
coeff = cpx.solution.sensitivity.objective() #Coeficientes
ld = cpx.solution.sensitivity.rhs() #lado derecho

#lista de variables
var_list = []
for i in range(20):
    for j in range(20):
        var = mdl.get_var_by_name('x_'+str(i)+'_'+str(j))
        var_list.append(var)

Sensibilidad de la función Objetivo

In [13]:
print(" ")
print(20*"-"+"Análisis - Sensibilidad Función Objetivo"+"-"*20)
print(" ")

for n in range(len(var_list)):
    print("La variable " + str(var_list[n]) + " " + str(coeff[n]))

 
--------------------Análisis - Sensibilidad Función Objetivo--------------------
 
La variable x_0_0 (0.0, 1e+20)
La variable x_0_1 (3103.03125, 1e+20)
La variable x_0_2 (2170.093750000001, 1e+20)
La variable x_0_3 (2474.3125, 1e+20)
La variable x_0_4 (2393.1875, 1e+20)
La variable x_0_5 (6530.562500000001, 1e+20)
La variable x_0_6 (1642.78125, 1e+20)
La variable x_0_7 (1581.9375, 1e+20)
La variable x_0_8 (1196.59375, 1e+20)
La variable x_0_9 (1115.46875, 1e+20)
La variable x_0_10 (2758.25, 1e+20)
La variable x_0_11 (3123.3125, 1e+20)
La variable x_0_12 (2717.6875, 1e+20)
La variable x_0_13 (2717.6875, 1e+20)
La variable x_0_14 (1602.21875, 1e+20)
La variable x_0_15 (2454.03125, 1e+20)
La variable x_0_16 (3691.1875, 1e+20)
La variable x_0_17 (2798.8125, 1e+20)
La variable x_0_18 (2170.09375, 1e+20)
La variable x_0_19 (1602.21875, 1e+20)
La variable x_1_0 (4522.71875, 5131.15625)
La variable x_1_1 (0.0, 1e+20)
La variable x_1_2 (2170.093750000001, 1e+20)
La variable x_1_3 (2474.3125, 

Restricciones y sensibilidad lado derecho

In [14]:
print(" ")
print(20*"-"+"Análisis - Restricciones y  Sensibilidad Lado Derecho"+"-"*20)
print(" ")

for n in range(n_restrict):
    print("La Restricción " + str(restricts[n]) + " " + str(ld[n]))

 
--------------------Análisis - Restricciones y  Sensibilidad Lado Derecho--------------------
 
La Restricción x_0_1+x_0_2+x_0_3+x_0_4+x_0_5+x_0_6+x_0_7+x_0_8+x_0_9+x_0_10+x_0_11+x_0_12+x_0_13+x_0_14+x_0_15+x_0_16+x_0_17+x_0_18+x_0_19 <= 4427.3099999999995 (0.0, 1e+20)
La Restricción x_1_0+x_1_2+x_1_3+x_1_4+x_1_5+x_1_6+x_1_7+x_1_8+x_1_9+x_1_10+x_1_11+x_1_12+x_1_13+x_1_14+x_1_15+x_1_16+x_1_17+x_1_18+x_1_19 <= 3373.47 (1405.18, 1e+20)
La Restricción x_2_0+x_2_1+x_2_3+x_2_4+x_2_5+x_2_6+x_2_7+x_2_8+x_2_9+x_2_10+x_2_11+x_2_12+x_2_13+x_2_14+x_2_15+x_2_16+x_2_17+x_2_18+x_2_19 <= 2137.74 (1101.71, 1e+20)
La Restricción x_3_0+x_3_1+x_3_2+x_3_4+x_3_5+x_3_6+x_3_7+x_3_8+x_3_9+x_3_10+x_3_11+x_3_12+x_3_13+x_3_14+x_3_15+x_3_16+x_3_17+x_3_18+x_3_19 <= 131.91 (61.31999999999999, 1537.0900000000001)
La Restricción x_4_0+x_4_1+x_4_2+x_4_3+x_4_5+x_4_6+x_4_7+x_4_8+x_4_9+x_4_10+x_4_11+x_4_12+x_4_13+x_4_14+x_4_15+x_4_16+x_4_17+x_4_18+x_4_19 <= 778.41 (483.71000000000015, 1e+20)
La Restricción x_5_0+x_5_1+x